In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.0`
import $ivy.`io.delta:delta-spark_2.13:3.1.0`

Downloaded https://repo1.maven.org/maven2/io/delta/delta-spark_2.13/3.1.0/delta-spark_2.13-3.1.0.pom
Downloaded https://repo1.maven.org/maven2/io/delta/delta-storage/3.1.0/delta-storage-3.1.0.pom


import $ivy.$
import $ivy.$

In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
  .getOrCreate()

import spark.implicits._

val tablePath = "../local/delta-scala"


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/30 17:55:50 WARN Utils: Your hostname, DESKTOP-CN01VS3 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/30 17:55:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/30 17:55:50 INFO SparkContext: Running Spark version 3.5.0
25/03/30 17:55:50 INFO SparkContext: OS info Linux, 5.15.167.4-microsoft-standard-WSL2, amd64
25/03/30 17:55:50 INFO SparkContext: Java version 17.0.14
25/03/30 17:55:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/30 17:55:51 INFO ResourceUtils: ==============================================================
25/03/30 17:55:51 INFO ResourceUtils: No custom resources configured for spark.driver.
25/03/30 17:55:51 INFO ResourceUtils: ==============================================================
25/03/30 17:55:51 INFO SparkCon

import org.apache.spark.sql.SparkSession
spark: SparkSession = org.apache.spark.sql.SparkSession@3c5dec56
import spark.implicits._
tablePath: String = "../local/delta-scala"

In [4]:
val df = Seq(
  (8, "bat"),
  (64, "mouse"),
  (1, "horse")
).toDF("number", "word")

df: org.apache.spark.sql.package.DataFrame = [number: int, word: string]

In [5]:
df.write
  .format("delta")
  .mode("overwrite")  // or append
  .save(tablePath)

25/03/30 17:56:17 INFO DelegatingLogStore: LogStore `LogStoreAdapter(io.delta.storage.HDFSLogStore)` is used for scheme `file`
25/03/30 17:56:17 INFO DeltaLog: Creating initial snapshot without metadata, because the directory is empty
25/03/30 17:56:17 INFO InitialSnapshot: [tableId=c797243d-c83b-4c8a-990c-9286606ef22f] Created snapshot InitialSnapshot(path=file:/home/eyal/code/local/delta-scala/_delta_log, version=-1, metadata=Metadata(3e1fa64d-34f7-42f3-ba5a-30f8988c5082,null,null,Format(parquet,Map()),null,List(),Map(),Some(1743346577157)), logSegment=LogSegment(file:/home/eyal/code/local/delta-scala/_delta_log,-1,List(),org.apache.spark.sql.delta.EmptyCheckpointProvider$@3bd7756,-1), checksumOpt=None)
25/03/30 17:56:17 INFO DeltaLog: No delta log found for the Delta table at file:/home/eyal/code/local/delta-scala/_delta_log
25/03/30 17:56:17 INFO InitialSnapshot: [tableId=3e1fa64d-34f7-42f3-ba5a-30f8988c5082] Created snapshot InitialSnapshot(path=file:/home/eyal/code/local/delta-sc

In [6]:
spark.read.format("delta").load(tablePath).show()

25/03/30 17:57:40 INFO PrepareDeltaScan: DELTA: Filtering files for query
25/03/30 17:57:40 INFO CodeGenerator: Code generated in 67.625965 ms
25/03/30 17:57:40 INFO CodeGenerator: Code generated in 10.440042 ms
25/03/30 17:57:40 INFO SparkContext: Starting job: $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128
25/03/30 17:57:40 INFO DAGScheduler: Got job 6 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128) with 50 output partitions
25/03/30 17:57:40 INFO DAGScheduler: Final stage: ResultStage 9 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128)
25/03/30 17:57:40 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 8)
25/03/30 17:57:40 INFO DAGScheduler: Missing parents: List()
25/03/30 17:57:40 INFO DAGScheduler: Submitting ResultStage 9 (MapPartitionsRDD[32] at $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128), which has no missing parents
25/03/30 17:57:40 INFO MemoryStore: Block broadc

+------+-----+
|number| word|
+------+-----+
|    64|mouse|
|     1|horse|
|     8|  bat|
+------+-----+

